# Dogs Image Classifcation
**The purposes of this notebook are:**

- Using a RESNET pre-trained net
- Applying various tools (augmentaiton, early stoppng etc.)

In [1]:
import neptune

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

# torchvision: popular datasets, model architectures, and common image transformations for computer vision.
import torchvision

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision import models
from torchsummary import summary

from sklearn.model_selection import train_test_split

import glob
import os.path as osp
import os
import numpy as np
from PIL import Image # to read images
import ntpath
# import pandas as pd

# import xml.etree.ElementTree as ET # for parsing XML
import matplotlib.pyplot as plt # to show images
import matplotlib.patches as patches
from tqdm import tqdm_notebook as tqdm
import time
%matplotlib inline

In [2]:
neptune.init('MM-IT-DOGS/dogs-project')

Project(MM-IT-DOGS/dogs-project)

In [133]:
PARAMS = {'random_seed' : 42,
          'batch_size' : 128,
          'learning_rate': 0.001,
          'momentum': 0.9,
          'n_epoch': 50,
          'optimizer': "adam",
          'fc_dropout': 0.4}

In [134]:
neptune.create_experiment(name='resnet_bs_256', 
                          params=PARAMS,
                          tags=['gcp', 'classification', 'adam', 'resnet', 'fixed_weights', 'lr_decay'],
                          description='Lower lr')

https://ui.neptune.ai/MM-IT-DOGS/dogs-project/e/SAN-75


Experiment(SAN-75)

In [135]:
BASE_PATH = "/home/ishay_telavivi/data"
#BASE_PATH = "/home/ishay/Documents/MDLI/dogs/data" #Ishay
#BASE_PATH = "g:\\My Drive\\AI\\MDLI #3 2019\\Projects\\dogs" #Michael
#IMGS_PATH = BASE_PATH + "\\Data\\Images\\" #Michael
#ANNTN_PATH = BASE_PATH + "\\Data\\Annotation\\" #Michael

In [136]:
CHECKPOINTS_DIR = '/home/ishay_telavivi/artifacts'

## Examine directory structure and content

In [137]:
print(os.listdir(BASE_PATH))

['Images', 'Annotation']


**Images**

In [138]:
IMGS_PATH = BASE_PATH + "/Images/" #Ishay
#breed_list = os.listdir(BASE_PATH + IMGS_PATH)[:5]

breed_list = os.listdir(IMGS_PATH)

print("Number of breeds:",len(breed_list))

Number of breeds: 120


## Create a Dataset and DataLoader Classes

The following code is based on the following links:

- https://pytorch.org/docs/stable/torchvision/datasets.html
- https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets
- https://discuss.pytorch.org/t/how-to-divide-dataset-into-training-validation-and-testing/38764/5

In [139]:
IMAGE_WIDTH = 224
IMAGE_LENGTH = 224

In [140]:
image_size_before_crop = 280

In [141]:
# Must at least transform to tensor
# We allow different transformation per dataset. For example we might want to crop the training, but leave the
# eval and test sets uncropped
transform = {"train": T.Compose([    
                T.Resize((image_size_before_crop,image_size_before_crop)),
                T.RandomCrop(IMAGE_WIDTH),
                T.RandomHorizontalFlip(),
                T.Resize((IMAGE_WIDTH,IMAGE_LENGTH)),
                T.ToTensor(),
                T.Normalize((0.4766, 0.4525, 0.3917), (0.2613, 0.2559, 0.2605)),
                ]),
             "eval": T.Compose([ 
                T.Resize((IMAGE_WIDTH,IMAGE_LENGTH)),
                T.ToTensor(),
                T.Normalize((0.4766, 0.4525, 0.3917), (0.2613, 0.2559, 0.2605)),
                ]),
             "test": T.Compose([ 
                T.Resize((IMAGE_WIDTH,IMAGE_LENGTH)),
                T.ToTensor(),
                 T.Normalize((0.4766, 0.4525, 0.3917), (0.2613, 0.2559, 0.2605)),
                ])}

In [142]:
# If we want to create datasets with different transformations for train/eval/test, we must create 3 identical
# sets, and then select the specific indices from each per data
train_data_set = ImageFolder(IMGS_PATH,transform=transform["train"])
eval_data_set = ImageFolder(IMGS_PATH,transform=transform["eval"])
test_data_set = ImageFolder(IMGS_PATH,transform=transform["test"])

In [143]:
# Now split the dataset

In [144]:
dataset_size = len(train_data_set) # This is not only the train as this phase, but the whole data.
dataset_size

20580

In [145]:
indices = list(range(dataset_size))

In [146]:
# Split the data like: {train: 0.7, eval: 0.2, test: 0.1}
train_split = 0.7
eval_split = 0.9 # (0.7 + 0.2)
shuffle_dataset = True

In [147]:
# Set the indices for split
split1 = int(np.floor(train_split * dataset_size))
split2 = int(np.floor(eval_split * dataset_size))
print(split1, split2)

14405 18522


In [148]:
if shuffle_dataset :
    np.random.seed(PARAMS["random_seed"])
    np.random.shuffle(indices)

In [149]:
train_indices, eval_indices, test_indices = indices[:split1], indices[split1:split2], indices[split2:]

In [150]:
print(len(train_indices), len(eval_indices), len(test_indices))

14405 4117 2058


In [151]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
eval_sampler = SubsetRandomSampler(eval_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [152]:
# Create the loaders
train_loader = DataLoader(train_data_set, batch_size=PARAMS["batch_size"], 
                                           sampler=train_sampler)
eval_loader = DataLoader(eval_data_set, batch_size=PARAMS["batch_size"],
                                                sampler=eval_sampler)
test_loader = DataLoader(test_data_set, batch_size=1,
                                                sampler=test_sampler)

## Use Pre-Trained Net

In [153]:
# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [154]:
pretrained_model = models.resnet50(pretrained=True)

In [155]:
# Here, we need to freeze all the network except the final layer. We need to set requires_grad == False to 
# freeze the parameters so that the gradients are not computed in backward()
for param in pretrained_model.parameters():
    param.requires_grad = False

In [156]:
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features

In [157]:
num_ftrs

2048

In [158]:
# Make sure that this is the final layer (that it output 1000 size tensor)
pretrained_model.fc.out_features

1000

In [159]:
# New we change the size of this last layer to output 120 size tensor
pretrained_model.fc = nn.Linear(num_ftrs, 120)

In [160]:
model = pretrained_model.to(device)

In [161]:
summary(pretrained_model, input_data=(3,IMAGE_WIDTH, IMAGE_LENGTH))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        (9,408)
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        (128)
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-1                   [-1, 256, 56, 56]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          (4,096)
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          (128)
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          (36,864)
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          (128)
|    |    └─ReLU: 3-6                  

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        (9,408)
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        (128)
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-1                   [-1, 256, 56, 56]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          (4,096)
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          (128)
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          (36,864)
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          (128)
|    |    └─ReLU: 3-6                  

In [162]:
if PARAMS["optimizer"]=="sgd":
    optimizer = optim.SGD(model.parameters(), lr=PARAMS["learning_rate"], momentum=PARAMS["momentum"])
elif PARAMS["optimizer"]=="adam":
    optimizer = optim.Adam(model.parameters(), lr=PARAMS["learning_rate"])
else:
    raise Exception('No defined optimzer')

In [163]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.08)

In [164]:
class EarlyStopping(object):
    def __init__(self, mode='max', min_delta=0, patience=3, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        # This is for the first round, before we have best
        if self.best is None:
            self.best = metrics
            return False
        # This is in case we have nan (for example if it explodes)
        if torch.isnan(metrics):
            return True
        # Standard - if metrics is better than the best, it zeros (start over)
        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        # Else, start counting till patience
        else:
            self.num_bad_epochs += 1
            print("Bad epochs:", self.num_bad_epochs)
        # If num_bad_epochs equals patience, stop (True)
        if self.num_bad_epochs >= self.patience:
            print("Terminate")
            return True
        

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [165]:
es = EarlyStopping(patience=4)

In [166]:
def train(model, optimizer, scheduler, epoch=1, log_interval=50):
    model.train()  # set training mode
    iteration = 0
    
    for ep in range(epoch):
        print("epoch {}".format(ep))
        print("========")
        for batch_idx, (data, target) in enumerate(train_loader):
            # bring data to the computing device, e.g. GPU
            data, target = data.to(device), target.to(device)

            # forward pass
            output = model(data)
            #print("output", output.size())
            #print("target", target.size())
            # compute loss
            loss = F.cross_entropy(output, target)
            
            # Clear all accumulated gradients
            optimizer.zero_grad()
            
            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            neptune.log_metric('batch_loss', loss.item())
            
#             if batch_idx % log_interval == 0:
#                 print('Iteration %d, loss = %.4f' % (batch_idx, loss.item()))
#                 test_acc = test(eval_loader, model)
#                 neptune.log_metric("batch_test_acc", test_acc)
#                 train_acc = test(train_loader, model, set_="train")
#                 neptune.log_metric("batch_train_acc", train_acc)
#                 print()
        # Since it takes a lot of time, I calculate only at epoc end        
        test_acc = test(eval_loader, model)
        train_acc = test(train_loader, model, set_="train")
        neptune.log_metric('loss', loss.item())
        neptune.log_metric('test_acc', test_acc)
        neptune.log_metric('train_acc', train_acc)
        
        # Decrease lr
        scheduler.step()
        
        # Saving checkpoing (first to local machine and then to neptune)
        torch.save(model.state_dict(),CHECKPOINTS_DIR+"/epoch_{}".format(ep))
        neptune.log_artifact(CHECKPOINTS_DIR+"/epoch_{}".format(ep))
        
        if es.step(torch.tensor(test_acc)):
            break  # early stop criterion is met, we can stop now

In [167]:
def test(loader, model, set_='eval'):
    model.eval()  # set evaluation mode
    num_correct = 0
    num_samples = 0
    test_loss = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, preds = output.max(1) # , keepdim=True)[1]
            num_correct += (preds == target).sum()
            num_samples += preds.size(0)
            
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        acc = float(num_correct) / num_samples
        print('{} Got {} / {} correct {:.2f}%'.format(set_, num_correct, num_samples, 100 * acc))
    test_loss /= len(loader.dataset)
    
    return acc

In [168]:
train(model, optimizer, exp_lr_scheduler, epoch=PARAMS["n_epoch"])

epoch 0
eval Got 3215 / 4117 correct 78.09%
train Got 11830 / 14405 correct 82.12%
epoch 1
eval Got 3355 / 4117 correct 81.49%
train Got 12542 / 14405 correct 87.07%
epoch 2
eval Got 3369 / 4117 correct 81.83%
train Got 12719 / 14405 correct 88.30%
epoch 3
eval Got 3386 / 4117 correct 82.24%
train Got 12942 / 14405 correct 89.84%
epoch 4
eval Got 3448 / 4117 correct 83.75%
train Got 13224 / 14405 correct 91.80%
epoch 5
eval Got 3463 / 4117 correct 84.11%
train Got 13222 / 14405 correct 91.79%
epoch 6
eval Got 3441 / 4117 correct 83.58%
train Got 13275 / 14405 correct 92.16%
Bad epochs: 1
epoch 7
eval Got 3447 / 4117 correct 83.73%
train Got 13320 / 14405 correct 92.47%
Bad epochs: 2
epoch 8
eval Got 3455 / 4117 correct 83.92%
train Got 13286 / 14405 correct 92.23%
Bad epochs: 3
epoch 9
eval Got 3453 / 4117 correct 83.87%
train Got 13280 / 14405 correct 92.19%
Bad epochs: 4
Terminate


In [169]:
neptune.stop()

## Try predicting on one image

Details about Varaible: https://jhui.github.io/2018/02/09/PyTorch-Variables-functionals-and-Autograd/

In [77]:
from torch.autograd import Variable

In [89]:
# image_path = '/home/ishay_telavivi/data/Images/n02085620-Chihuahua/n02085620_1073.jpg'
image_path = '/home/ishay_telavivi/data/Images/n02106030-collie/n02106030_7977.jpg'

In [90]:
trasformer = transform["eval"]

In [91]:
def predict_image_path(path, transformer, model, image_width=IMAGE_WIDTH, image_length=IMAGE_LENGTH):
    """
    Receives an image path and returns the class (breed)
    """
    
    image = Image.open(path)
    trans_image = transformer(image).float()
    image_var = Variable(trans_image, requires_grad=True)
    image_for_pred = image_var.view(1,3,image_width,image_length)
    image_for_pred_to_device = image_for_pred.to(device)
    
    model.eval()
    output = model(image_for_pred_to_device)
    
    indices = torch.argmax(output, 1)
    breed = test_loader.dataset.classes[indices]
    
    return breed

In [92]:
predict_image_path(image_path,
                  trasformer,
                  model)

'n02106030-collie'